## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-30-05-31-55 +0000,wapo,China sentences 11 criminal gang leaders to de...,https://www.washingtonpost.com/world/2025/09/3...
1,2025-09-30-05-30-00 +0000,wsj,Coty Considers Sale or Spinoff of CoverGirl,https://www.wsj.com/business/retail/coty-consi...
2,2025-09-30-05-13-00 +0000,wsj,China’s Factory Activity Gauges Show Signs of ...,https://www.wsj.com/economy/chinas-manufacturi...
3,2025-09-30-05-01-07 +0000,bbc,"New online NHS hospital service by 2027, PM to...",https://www.bbc.com/news/articles/c9317k58xkjo...
4,2025-09-30-05-00-55 +0000,cbc,"Nearly 100 children hurt, several feared dead ...",https://www.cbc.ca/news/world/indonesia-school...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2311/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
128,trump,54
47,government,18
70,shutdown,17
19,new,16
129,gaza,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
119,2025-09-29-21-14-29 +0000,nypost,Trump unveils 20-point Gaza cease-fire plan wi...,https://nypost.com/2025/09/29/us-news/trump-un...,137
21,2025-09-30-03-05-00 +0000,wsj,Top Democrats made their case to Trump for res...,https://www.wsj.com/politics/policy/government...,134
89,2025-09-29-22-29-55 +0000,nypost,Ex-UK PM Tony Blair to help run Gaza under Tru...,https://nypost.com/2025/09/29/us-news/ex-uk-pm...,114
71,2025-09-29-23-27-45 +0000,nyt,Trump and Netanyahu Tell Hamas to Accept Gaza ...,https://www.nytimes.com/2025/09/29/us/politics...,106
131,2025-09-29-20-50-55 +0000,nyt,Read Each Point of Trump’s Plan for an Israel-...,https://www.nytimes.com/2025/09/29/world/middl...,104


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
119,137,2025-09-29-21-14-29 +0000,nypost,Trump unveils 20-point Gaza cease-fire plan wi...,https://nypost.com/2025/09/29/us-news/trump-un...
21,80,2025-09-30-03-05-00 +0000,wsj,Top Democrats made their case to Trump for res...,https://www.wsj.com/politics/policy/government...
310,63,2025-09-29-09-00-00 +0000,wsj,"Rahm Emanuel, former Democratic congressman, W...",https://www.wsj.com/politics/policy/rahm-emanu...
158,50,2025-09-29-18-55-45 +0000,nypost,Frank founder Charlie Javice sobs in court as ...,https://nypost.com/2025/09/29/business/charlie...
195,43,2025-09-29-16-49-04 +0000,cbc,Police continue to search for motive in deadly...,https://www.cbc.ca/news/canada/windsor/deadly-...
236,42,2025-09-29-14-37-00 +0000,wsj,"Eric Adams Ends Re-Election Bid, Reshaping Rac...",https://www.wsj.com/us-news/new-york-city-mayo...
299,41,2025-09-29-09-30-00 +0000,wsj,"The Pentagon, alarmed at the low weapons stock...",https://www.wsj.com/politics/national-security...
64,40,2025-09-30-00-01-00 +0000,wsj,American Eagle Outfitters’ chief executive had...,https://www.wsj.com/business/retail/you-cant-r...
129,37,2025-09-29-20-53-00 +0000,wsj,The Trump administration said it is cracking d...,https://www.wsj.com/politics/national-security...
265,35,2025-09-29-12-05-00 +0000,wsj,Moldova’s Pro-Europe Party Claims Victory Over...,https://www.wsj.com/world/europe/moldovas-pro-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
